In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import math

import cv2
import numpy as np
import scipy
from scipy import ndimage, spatial

import transformations


def inbounds(shape, indices):
    assert len(shape) == len(indices)
    for i, ind in enumerate(indices):
        if ind < 0 or ind >= shape[i]:
            return False
    return True

In [3]:
class KeypointDetector(object):
    def detectKeypoints(self, image):
        '''
        Input:
            image -- uint8 BGR image with values between [0, 255]
        Output:
            list of detected keypoints, fill the cv2.KeyPoint objects with the
            coordinates of the detected keypoints, the angle of the gradient
            (in degrees), the detector response (Harris score for Harris detector)
            and set the size to 10.
        '''
        raise NotImplementedError()


class DummyKeypointDetector(KeypointDetector):
    '''
    Compute silly example features. This doesn't do anything meaningful, but
    may be useful to use as an example.
    '''

    def detectKeypoints(self, image):
        '''
        Input:
            image -- uint8 BGR image with values between [0, 255]
        Output:
            list of detected keypoints, fill the cv2.KeyPoint objects with the
            coordinates of the detected keypoints, the angle of the gradient
            (in degrees), the detector response (Harris score for Harris detector)
            and set the size to 10.
        '''
        image = image.astype(np.float32)
        image /= 255.
        features = []
        height, width = image.shape[:2]

        for y in range(height):
            for x in range(width):
                r = image[y, x, 0]
                g = image[y, x, 1]
                b = image[y, x, 2]

                if int(255 * (r + g + b) + 0.5) % 100 == 1:
                    # If the pixel satisfies this meaningless criterion,
                    # make it a feature.

                    f = cv2.KeyPoint()
                    f.pt = (x, y)
                    # Dummy size
                    f.size = 10
                    f.angle = 0
                    f.response = 10

                    features.append(f)

        return features


class HarrisKeypointDetector(KeypointDetector):

    # Compute harris values of an image.
    def computeHarrisValues(self, srcImage):
        '''
        Input:
            srcImage -- Grayscale input image in a numpy array with
                        values in [0, 1]. The dimensions are (rows, cols).
        Output:
            harrisImage -- numpy array containing the Harris score at
                           each pixel.
            orientationImage -- numpy array containing the orientation of the
                                gradient at each pixel in degrees.
        '''
        height, width = srcImage.shape[:2]

        harrisImage = np.zeros(srcImage.shape[:2])
        orientationImage = np.zeros(srcImage.shape[:2])

        # TODO 1: Compute the harris corner strength for 'srcImage' at
        # each pixel and store in 'harrisImage'.  See the project page
        # for direction on how to do this. Also compute an orientation
        # for each pixel and store it in 'orientationImage.'
        # TODO-BLOCK-BEGIN
        
        # Compute the x, y derivatives
        i_x = ndimage.sobel(srcImage, 1)
        i_y = ndimage.sobel(srcImage, 0)
        
        i_xx = i_x ** 2
        i_yy = i_y ** 2
        i_xy = i_x * i_y
        
        # Gauss mask
        gauss_xx = ndimage.gaussian_filter(i_xx, 0.5)
        gauss_yy = ndimage.gaussian_filter(i_yy, 0.5)
        gauss_xy = ndimage.gaussian_filter(i_xy, 0.5)
        
        for x in range(height):
            for y in range(width):
                H = np.array([[gauss_xx[x, y], gauss_xy[x, y]], [gauss_xy[x, y], gauss_yy[x, y]]], dtype = "float")
                harrisImage[x, y] = np.linalg.det(H) - 0.1 * np.trace(H) ** 2
                orientationImage[x, y] = np.degrees(np.arctan2(i_y[x, y], i_x[x, y]))
        # TODO-BLOCK-END

        return harrisImage, orientationImage

    def computeLocalMaxima(self, harrisImage):
        '''
        Input:
            harrisImage -- numpy array containing the Harris score at
                           each pixel.
        Output:
            destImage -- numpy array containing True/False at
                         each pixel, depending on whether
                         the pixel value is the local maxima in
                         its 7x7 neighborhood.
        '''
        #destImage = np.zeros_like(harrisImage, np.bool)

        # TODO 2: Compute the local maxima image
        # TODO-BLOCK-BEGIN
        maxima = ndimage.filters.maximum_filter(harrisImage, (7, 7))
        destImage = maxima - harrisImage
        # TODO-BLOCK-END

        return destImage == 0

    def detectKeypoints(self, image):
        '''
        Input:
            image -- BGR image with values between [0, 255]
        Output:
            list of detected keypoints, fill the cv2.KeyPoint objects with the
            coordinates of the detected keypoints, the angle of the gradient
            (in degrees), the detector response (Harris score for Harris detector)
            and set the size to 10.
        '''
        image = image.astype(np.float32)
        image /= 255.
        height, width = image.shape[:2]
        features = []

        # Create grayscale image used for Harris detection
        grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # computeHarrisValues() computes the harris score at each pixel
        # position, storing the result in harrisImage.
        # You will need to implement this function.
        harrisImage, orientationImage = self.computeHarrisValues(grayImage)

        # Compute local maxima in the Harris image.  You will need to
        # implement this function. Create image to store local maximum harris
        # values as True, other pixels False
        harrisMaxImage = self.computeLocalMaxima(harrisImage)

        # Loop through feature points in harrisMaxImage and fill in information
        # needed for descriptor computation for each point.
        # You need to fill x, y, and angle.
        for y in range(height):
            for x in range(width):
                if not harrisMaxImage[y, x]:
                    continue

                f = cv2.KeyPoint()

                # TODO 3: Fill in feature f with location and orientation
                # data here. Set f.size to 10, f.pt to the (x,y) coordinate,
                # f.angle to the orientation in degrees and f.response to
                # the Harris score
                # TODO-BLOCK-BEGIN
                f.size = 10
                f.pt = (x, y)
                f.angle = orientationImage[y, x]
                f.response = harrisImage[y, x]
                # TODO-BLOCK-END

                features.append(f)

        return features

In [4]:
import math
import numpy as np


def get_rot_mx(angle_x, angle_y, angle_z):
    '''
    Input:
        angle_x -- Rotation around the x axis in radians
        angle_y -- Rotation around the y axis in radians
        angle_z -- Rotation around the z axis in radians
    Output:
        A 4x4 numpy array representing 3D rotations. The order of the rotation
        axes from first to last is x, y, z, if you multiply with the resulting
        rotation matrix from left.
    '''
    # Note: For MOPS, you need to use angle_z only, since we are in 2D

    rot_x_mx = np.array([[1, 0, 0, 0],
                         [0, math.cos(angle_x), -math.sin(angle_x), 0],
                         [0, math.sin(angle_x), math.cos(angle_x), 0],
                         [0, 0, 0, 1]])

    rot_y_mx = np.array([[math.cos(angle_y), 0, math.sin(angle_y), 0],
                         [0, 1, 0, 0],
                         [-math.sin(angle_y), 0, math.cos(angle_y), 0],
                         [0, 0, 0, 1]])

    rot_z_mx = np.array([[math.cos(angle_z), -math.sin(angle_z), 0, 0],
                         [math.sin(angle_z), math.cos(angle_z), 0, 0],
                         [0, 0, 1, 0],
                         [0, 0, 0, 1]])

    return np.dot(rot_z_mx, np.dot(rot_y_mx, rot_x_mx))


def get_trans_mx(trans_vec):
    '''
    Input:
        trans_vec -- Translation vector represented by an 1D numpy array with 3
        elements
    Output:
        A 4x4 numpy array representing 3D translation.
    '''
    assert trans_vec.ndim == 1
    assert trans_vec.shape[0] == 3

    trans_mx = np.eye(4)
    trans_mx[:3, 3] = trans_vec

    return trans_mx


def get_scale_mx(s_x, s_y, s_z):
    '''
    Input:
        s_x -- Scaling along the x axis
        s_y -- Scaling along the y axis
        s_z -- Scaling along the z axis
    Output:
        A 4x4 numpy array representing 3D scaling.
    '''
    # Note: For MOPS, you need to use s_x and s_y only, since we are in 2D
    scale_mx = np.eye(4)

    for i, s in enumerate([s_x, s_y, s_z]):
        scale_mx[i, i] = s

    return scale_mx



In [5]:
class FeatureDescriptor(object):
    # Implement in child classes
    def describeFeatures(self, image, keypoints):
        '''
        Input:
            image -- BGR image with values between [0, 255]
            keypoints -- the detected features, we have to compute the feature
            descriptors at the specified coordinates
        Output:
            Descriptor numpy array, dimensions:
                keypoint number x feature descriptor dimension
        '''
        raise NotImplementedError

In [189]:
class MOPSFeatureDescriptor(FeatureDescriptor):
    # TODO: Implement parts of this function
    def describeFeatures(self, image, keypoints):
        '''
        Input:
            image -- BGR image with values between [0, 255]
            keypoints -- the detected features, we have to compute the feature
            descriptors at the specified coordinates
        Output:
            desc -- K x W^2 numpy array, where K is the number of keypoints
                    and W is the window size
        '''
        image = image.astype(np.float32)
        image /= 255.
        # This image represents the window around the feature you need to
        # compute to store as the feature descriptor (row-major)
        windowSize = 8
        desc = np.zeros((len(keypoints), windowSize * windowSize))
        grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        grayImage = ndimage.gaussian_filter(grayImage, 0.5)
        for i, f in enumerate(keypoints):
            # TODO 5: Compute the transform as described by the feature
            # location/orientation. You will need to compute the transform
            # from each pixel in the 40x40 rotated window surrounding
            # the feature to the appropriate pixels in the 8x8 feature
            # descriptor image.
            transMx = np.zeros((2, 3))

            # TODO-BLOCK-BEGIN
            # Get axis
            x, y = f.pt
            temp_pt = np.array([-x, -y, 0])
            # Get response
            respon = f.response
            # Get angle back in rad
            rad_angle = np.deg2rad(f.angle)
            # Get rotation: For MOPS, you need to use angle_z only, since we are in 2D
            rotations = get_rot_mx(0, 0, -rad_angle)[:,[0, 1, 3]][[0, 1, 3], :]
            # Get translation matrix
            trans = get_trans_mx(temp_pt)[:,[0, 1, 3]][[0, 1, 3], :]
            # Get scale down by 5
            scales = get_scale_mx(1/5, 1/5, 0)[:,[0, 1, 3]][[0, 1, 3], :]
            # translation 2 to match axis
            trans_2 = get_trans_mx(np.array([4,4,0]))[:, [0, 1, 3]][[0, 1, 3], :]
            # Dot them
            dot_tran = trans_2.dot(scales).dot(rotations).dot(trans)
            transMx = dot_tran[:2,:]

            # TODO-BLOCK-END

            # Call the warp affine function to do the mapping
            # It expects a 2x3 matrix
            destImage = cv2.warpAffine(grayImage, transMx,
                (windowSize, windowSize), flags=cv2.INTER_LINEAR)
            
            # TODO 6: Normalize the descriptor to have zero mean and unit 
            # variance. If the variance is negligibly small (which we 
            # define as less than 1e-10) then set the descriptor
            # vector to zero. Lastly, write the vector to desc.
            # TODO-BLOCK-BEGIN
            image_mean = np.mean(destImage)
            image_std = np.std(destImage)
            if image_std ** 2 > 1e-10:
                desc[i] = ((np.array(destImage) - image_mean) / image_std).flatten()
            else:
                desc[i] = np.zeros(windowSize * windowSize)
        return desc

In [190]:
p = MOPSFeatureDescriptor()
what0 = p.describeFeatures(image_0, ff0)

In [7]:
class FeatureMatcher(object):
    def matchFeatures(self, desc1, desc2):
        '''
        Input:
            desc1 -- the feature descriptors of image 1 stored in a numpy array,
                dimensions: rows (number of key points) x
                columns (dimension of the feature descriptor)
            desc2 -- the feature descriptors of image 2 stored in a numpy array,
                dimensions: rows (number of key points) x
                columns (dimension of the feature descriptor)
        Output:
            features matches: a list of cv2.DMatch objects
                How to set attributes:
                    queryIdx: The index of the feature in the first image
                    trainIdx: The index of the feature in the second image
                    distance: The distance between the two features
        '''
        raise NotImplementedError

    # Evaluate a match using a ground truth homography.  This computes the
    # average SSD distance between the matched feature points and
    # the actual transformed positions.
    @staticmethod
    def evaluateMatch(features1, features2, matches, h):
        d = 0
        n = 0

        for m in matches:
            id1 = m.queryIdx
            id2 = m.trainIdx
            ptOld = np.array(features2[id2].pt)
            ptNew = FeatureMatcher.applyHomography(features1[id1].pt, h)

            # Euclidean distance
            d += np.linalg.norm(ptNew - ptOld)
            n += 1

        return d / n if n != 0 else 0

    # Transform point by homography.
    @staticmethod
    def applyHomography(pt, h):
        x, y = pt
        d = h[6]*x + h[7]*y + h[8]

        return np.array([(h[0]*x + h[1]*y + h[2]) / d,
            (h[3]*x + h[4]*y + h[5]) / d])

In [122]:
class RatioFeatureMatcher(FeatureMatcher):
    def matchFeatures(self, desc1, desc2):
        '''
        Input:
            desc1 -- the feature descriptors of image 1 stored in a numpy array,
                dimensions: rows (number of key points) x
                columns (dimension of the feature descriptor)
            desc2 -- the feature descriptors of image 2 stored in a numpy array,
                dimensions: rows (number of key points) x
                columns (dimension of the feature descriptor)
        Output:
            features matches: a list of cv2.DMatch objects
                How to set attributes:
                    queryIdx: The index of the feature in the first image
                    trainIdx: The index of the feature in the second image
                    distance: The ratio test score
        '''
        matches = []
        # feature count = n
        assert desc1.ndim == 2
        # feature count = m
        assert desc2.ndim == 2
        # the two features should have the type
        assert desc1.shape[1] == desc2.shape[1]

        if desc1.shape[0] == 0 or desc2.shape[0] == 0:
            return []

        # TODO 8: Perform ratio feature matching.
        # This uses the ratio of the SSD distance of the two best matches
        # and matches a feature in the first image with the closest feature in the
        # second image.
        # Note: multiple features from the first image may match the same
        # feature in the second image.
        # You don't need to threshold matches in this function
        # TODO-BLOCK-BEGIN
        
        # Check every feature in first image
        for i in range(desc1.shape[0]):
            # Take one feature from the first image
            first_part = [desc1[i]]
            # Calculate distance from all features in second image
            dist = scipy.spatial.distance.cdist(first_part, desc2)[0]
            # Sort
            sorted_dist = np.argsort(dist)
            # Find the index of the closest and second closest feature
            min_1 = sorted_dist[0]
            min_2 = sorted_dist[1]
            # Calculate the ratio
            if dist[min_1] == 0 and dist[min_2] == 0:
                r = 0
            else:
                r = dist[min_1] / dist[min_2]
            matches.append(cv2.DMatch(i, min_1, r))
        # TODO-BLOCK-END
        return matches

In [25]:
image_0 = cv2.imread("resources/triangle1.jpg")
image_0 = cv2.cvtColor(image_0, cv2.COLOR_BGR2RGB)
image_1 = cv2.imread("resources/triangle2.jpg")
image_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2RGB)

In [26]:
c = HarrisKeypointDetector()
ff0 = c.detectKeypoints(image_0)
ff1 = c.detectKeypoints(image_1)

In [15]:
p = MOPSFeatureDescriptor()
what0 = p.describeFeatures(image_0, ff0)
what1 = p.describeFeatures(image_1, ff1)

In [123]:
m = RatioFeatureMatcher()
res = m.matchFeatures(what0, what1)